# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [1]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [2]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
import re
import os
import pytest
import io
import contextlib
from bs4 import BeautifulSoup
from typing import List, Dict


## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [3]:
def get_book_data(book_url: str) -> dict:
    """
    МЕСТО ДЛЯ ДОКУМЕНТАЦИИ
    Получает подробную информацию о книге с указанной страницы.

    Функция парсит HTML-страницу книги, извлекает основные данные (название, цена, рейтинг,
    наличие, описание) и дополнительную информацию из таблицы Product Information.

    Args:
        book_url (str): URL-адрес страницы с книгой для парсинга

    Returns:
        dict: Словарь с данными о книге, содержащий следующие ключи:
            - 'title': название книги (str)
            - 'price': цена в формате строки (str)
            - 'rating': рейтинг в виде числа от 1 до 5 (int)
            - 'stock': количество книг в наличии (int)
            - 'description': описание книги (str)
            - 'upc': универсальный код товара (str)
            - 'product_type': тип продукта (str)
            - 'price_excl_tax': цена без учета налогов (str)
            - 'price_incl_tax': цена с учетом налогов (str)
            - 'tax': сумма налога (str)
            - 'availability': информация о доступности (str)
            - 'num_reviews': количество отзывов (int)
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ

    try:
        # Отправляем GET-запрос к странице книги
        response = requests.get(book_url)
        response.raise_for_status()  # Проверяем успешность запроса

        # Создаем объект BeautifulSoup для парсинга HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Извлекаем основную информацию о книге
        title = soup.find('h1').text.strip()

        # Извлекаем цену
        price = soup.find('p', class_='price_color').text.strip()

        # Извлекаем рейтинг (преобразуем классы звезд в числовой формат)
        rating_element = soup.find('p', class_='star-rating')
        rating_classes = rating_element.get('class', [])
        rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
        rating = 0
        for class_name in rating_classes:
            if class_name in rating_map:
                rating = rating_map[class_name]
                break

        # Извлекаем информацию о наличии
        stock_text = soup.find('p', class_='instock availability').text.strip()
        stock_match = re.search(r'(\d+)', stock_text)
        stock = int(stock_match.group(1)) if stock_match else 0

        # Извлекаем описание
        description_element = soup.find('div', id='product_description')
        description = ""
        if description_element:
            description_sibling = description_element.find_next_sibling('p')
            if description_sibling:
                description = description_sibling.text.strip()
        product_info = {}
        info_table = soup.find('table', class_='table table-striped')
        if info_table:
            rows = info_table.find_all('tr')
            for row in rows:
                header = row.find('th').text.strip()
                value = row.find('td').text.strip()
                product_info[header] = value
        book_data = {
            'title': title,
            'price': price,
            'rating': rating,
            'stock': stock,
            'description': description,
            'upc': product_info.get('UPC', ''),
            'product_type': product_info.get('Product Type', ''),
            'price_excl_tax': product_info.get('Price (excl. tax)', ''),
            'price_incl_tax': product_info.get('Price (incl. tax)', ''),
            'tax': product_info.get('Tax', ''),
            'availability': product_info.get('Availability', ''),
            'num_reviews': int(product_info.get('Number of reviews', 0))
        }

        return book_data

    except requests.RequestException as e:
        print(f"Ошибка при запросе к странице: {e}")
        return {}
    except Exception as e:
        print(f"Ошибка при парсинге данных: {e}")
        return {}
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [4]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': '£51.77',
 'rating': 3,
 'stock': 22,
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up the

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [5]:
def scrape_books(is_save: bool = False):
    """
    МЕСТО ДЛЯ ДОКУМЕНТАЦИИ
    Собирает данные обо всех книгах из каталога сайта Books to Scrape.

    Функция проходит по всем страницам каталога, извлекает ссылки на отдельные книги
    и использует функцию get_book_data для сбора информации о каждой книге.

    Args:
        is_save (bool): Флаг сохранения результатов в файл. Если True, данные
                       сохраняются в файл 'books_data.txt', иначе пропускается.

    Returns:
        list: Список словарей с данными о всех книгах
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    base_url = "http://books.toscrape.com/catalogue/"
    all_books_data = []
    page_number = 1

    print("Начало сбора данных о книгах...")

    while True:
        if page_number == 1:
            catalog_url = f"{base_url}index.html"
        else:
            catalog_url = f"{base_url}page-{page_number}.html"

        try:
            print(f"Обрабатывается страница {page_number}...")
            response = requests.get(catalog_url)

            if response.status_code == 404:
                print("Достигнута последняя страница каталога.")
                break

            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            book_elements = soup.find_all('article', class_='product_pod')

            if not book_elements:
                print("На странице не найдено книг.")
                break
            book_urls = []
            for book_element in book_elements:
                link_element = book_element.find('h3').find('a')
                if link_element and link_element.get('href'):
                    book_relative_url = link_element['href']
                    if book_relative_url.startswith('../../../'):
                        book_id = book_relative_url.replace('../../../', '')
                        book_full_url = f"{base_url}{book_id}"
                    else:
                        book_full_url = f"{base_url}{book_relative_url}"
                    book_urls.append(book_full_url)

            for i, book_url in enumerate(book_urls, 1):
                print(f"  Обрабатывается книга {i}/{len(book_urls)}")
                book_data = get_book_data(book_url)

                if book_data:
                    all_books_data.append(book_data)
                    print(f"    ✓ Собраны данные: {book_data['title'][:50]}...")
                else:
                    print(f"    ✗ Ошибка при сборе данных")
                time.sleep(0.3)
            page_number += 1
            time.sleep(0.5)

        except requests.RequestException as e:
            print(f"Ошибка при загрузке страницы {page_number}: {e}")
            break
        except Exception as e:
            print(f"Неожиданная ошибка на странице {page_number}: {e}")
            break

    print(f"\nСбор данных завершен. Обработано книг: {len(all_books_data)}")

    if is_save and all_books_data:
        try:
            filename = 'books_data.txt'
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(f"Всего книг: {len(all_books_data)}\n")
                file.write("=" * 80 + "\n\n")

                for i, book in enumerate(all_books_data, 1):
                    file.write(f"Книга #{i}\n")
                    file.write("-" * 40 + "\n")

                    for key, value in book.items():
                        file.write(f"{key}: {value}\n")

                    file.write("\n" + "=" * 80 + "\n\n")

            print(f"✓ Данные сохранены в файл: {filename}")
            print(f"Путь к файлу: {os.path.abspath(filename)}")

        except Exception as e:
            print(f"✗ Ошибка при сохранении в файл: {e}")

    return all_books_data


    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [6]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

Начало сбора данных о книгах...
Обрабатывается страница 1...
Достигнута последняя страница каталога.

Сбор данных завершен. Обработано книг: 0
<class 'list'> 0


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [7]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
def run_daily_at_nineteen():

    print("Настройка ежедневного сбора данных в 19:00...")

    # Настраиваем расписание
    schedule.every().day.at("19:00").do(scheduled_scraping)

    print("Планировщик запущен. Ожидание времени выполнения...")
    print("Текущее время:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print("Следующее выполнение: сегодня в 19:00")
    print("Для остановки нажмите Ctrl+C")

    try:
        while True:
            schedule.run_pending()
            time.sleep(60)
    except KeyboardInterrupt:
        print("\nПланировщик остановлен")

# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [8]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest test/test_scraper.py

============================= test session starts ==============================
platform linux -- Python 3.12.12, pytest-8.4.2, pluggy-1.6.0
rootdir: /content
plugins: anyio-4.11.0, typeguard-4.4.4, langsmith-0.4.40
collected 0 items                                                              

============================ no tests ran in 0.01s =============================
ERROR: file or directory not found: test/test_scraper.py



## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```